In [5]:
import bitfinex

import datetime
import time
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [6]:
%config IPCompleter.use_jedi = False

----------

* [ ] implementare check ultima data csv di ogni pair per scaricare solamente gli ultimi giorni mancanti.

Bitfinex gestisce il tempo in relazione al **UTC** (*Universal Time Convention*), il nostro è chiaramente UTC+1.

In [43]:
api_v1 = bitfinex.bitfinex_v1.api_v1()
pairs = np.array(api_v1.symbols())
#pairs = pairs[[(pair[-3:]=='usd' and pair[-4]!=':') for pair in pairs]]

No keys, only access to public API functions


In [113]:
save_path = './data/'
api_v2 = bitfinex.bitfinex_v2.api_v2()

for pair in pairs:
    try:
        pair_data = pd.read_csv(save_path + pair + '.csv')
        exist = True
    except:
        print(f'Pair {pair} does not exist.' )
        exist = False
    
    if exist:
        start = pd.to_datetime(pair_data.time.values[-1], unit='ms')
        start = date_2_stamp(start)
        
        part_data = hist_data(api_v2, symb=pair, start=start)
        ind = [np.ndim(x) != 0 for x in part_data]
        part_data = [i for (i, v) in zip(part_data, ind) if v]
        
        names = ['time', 'open', 'close', 'high', 'low', 'volume']
        part_data = pd.DataFrame(part_data, columns=names)
        part_data.drop_duplicates(inplace=True)
               
        pair_data = pair_data.append(part_data)

        print('Download complete! Saving to CSV format..')
        pair_data.to_csv(save_path + pair + '.csv')
        print('Done! Moving to next pair.\n')
        
    else:
        next

print('END')

No keys, only access to public API functions
Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..
Done! Moving to next pair.

Download complete! Saving to CSV format..


KeyboardInterrupt: 

In [110]:
def load_pair(pair, path='./data/'):
    df = pd.read_csv(path + pair + '.csv')
    
    dt = pd.to_datetime(df['time'], unit='ms')
    
    df.set_index(dt, inplace=True)
    df.drop(columns='time', inplace=True)
    df.sort_index(inplace=True)
    
    return df

----

## Methods declaration

In [34]:
def date_2_stamp(date):
    '''
    Convert UTC date format to unix timestamp.
    
    Parameters
    ----------
    datetime : string
        Object indicating desired date to convert. Suggested format (YYYY-MM-DD).
    '''
    
    stamp = pd.to_datetime(pd.to_datetime(date))
    stamp = ((stamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s'))*1000
    
    return stamp

In [100]:
def hist_data(api_v2, symb='btcusd', start=1364767200000, stop='today', interval='1m', tick_limit=1000, step=60000000):

    if stop == 'today':
        today = datetime.datetime.today()
        stop = time.mktime(today.timetuple())*1000
    
    data = []
    
    print(f'Downloading {pair.upper()}..')
    year = datetime.datetime.fromtimestamp(start/1000).year

    while start < stop:          
        end = start + step
        res = api_v2.candles(symbol=symb, interval=interval, limit=tick_limit, start=start, end=end)
        data.extend(res)
        
        new_year = datetime.datetime.fromtimestamp(end/1000).year
        if new_year > year:
            print(f'{year} completed!')
            year = new_year

        start += step
        time.sleep(1.5)

    return data

---